In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection as ms

Εισαγωγή βιβλιοθηκών της Python που θα χρησιμοποιηθούν.

In [0]:
upl = files.upload()
Train_Data = pd.read_csv('train_dataSFLCUT2.csv', header = None)
Train_Labels = pd.read_csv('train_labelsSFL.csv', header = None)
Test_Data = pd.read_csv('test_dataSFLCUT2.csv', header = None)
Test_Labels = pd.read_csv('test_labelsSFL.csv', header = None)

Έγινε η εισαγωγή των csv αρχείων του manual χωρισμού train/test.

In [0]:
upl = files.upload()
Data = pd.read_csv('dataSFLCUT2.csv', header = None)
Labels = pd.read_csv('labelsSFL.csv', header = None)

Έγινε η εισαγωγή των csv αρχείων για το cross-validation.

In [0]:
def plot_history(histories, key='acc'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Validate')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.title()+'uracy')
  plt.legend()

  plt.xlim([0,max(history.epoch)])

Συνάρτηση σχεδίασης γραφικής παράστασης απόδοσης.

In [0]:
model_ini = keras.Sequential()

model_ini.add(keras.layers.Dense(units = 64, activation = tf.nn.relu, input_dim = 68))
model_ini.add(keras.layers.Dense(units = 32, activation = tf.nn.relu))
model_ini.add(keras.layers.Dense(units = 16, activation = tf.nn.relu))
model_ini.add(keras.layers.Dense(units = 8, activation = tf.nn.softmax))

model_ini.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

ini_history = model_ini.fit(Train_Data, Train_Labels, epochs = 100, validation_data = (Test_Data, Test_Labels))

plot_history([('Initial', ini_history)])

Αρχικό μοντέλο.

In [0]:
model_smaller = keras.Sequential()

model_smaller.add(keras.layers.Dense(units = 32, activation = tf.nn.relu, input_dim = 68))
model_smaller.add(keras.layers.Dense(units = 16, activation = tf.nn.relu))
model_smaller.add(keras.layers.Dense(units = 8, activation = tf.nn.softmax))

model_smaller.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

smaller_history = model_smaller.fit(Train_Data, Train_Labels, epochs = 100, validation_data = (Test_Data, Test_Labels))

plot_history([('Initial', ini_history), ('Smaller', smaller_history)])

Μικρότερο μοντέλο.

In [0]:
model_overmin = keras.Sequential()

model_overmin.add(keras.layers.Dense(units = 32, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu, input_dim = 68))
model_overmin.add(keras.layers.Dropout(0.2))
model_overmin.add(keras.layers.Dense(units = 16, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu))
model_overmin.add(keras.layers.Dropout(0.2))
model_overmin.add(keras.layers.Dense(units = 8, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.softmax))

model_overmin.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

overmin_history = model_overmin.fit(Train_Data, Train_Labels, epochs = 100, validation_data = (Test_Data, Test_Labels))

plot_history([('Initial', ini_history), ('Smaller', smaller_history), ('Regularizers and Dropout', overmin_history)])

Μοντέλο με regularizers (πάνω στο μικρότερο).

In [0]:
model_relu = keras.Sequential()

model_relu.add(keras.layers.Dense(units = 32, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.relu, input_dim = 32))
model_relu.add(keras.layers.Dropout(0.2))
model_relu.add(keras.layers.Dense(units = 16, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.relu))
model_relu.add(keras.layers.Dropout(0.2))
model_relu.add(keras.layers.Dense(units = 8, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.softmax))

model_relu.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

relu_history = model_relu.fit(Train_Data, Train_Labels, epochs = 1000, validation_data = (Test_Data, Test_Labels))

plot_history([('Subset feature vector', relu_history)])

Μοντέλο με regularizers ξανά, αλλά για το υποσύνολο των χαρακτηριστικών που επιλέχθηκε (16 χαρακτηριστικά, σε μέση τιμή και τυπική απόκλιση = 32 τιμές).

In [0]:
model_selu = keras.Sequential()

model_selu.add(keras.layers.Dense(units = 32, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu, input_dim = 32))
model_selu.add(keras.layers.Dropout(0.2))
model_selu.add(keras.layers.Dense(units = 16, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu))
model_selu.add(keras.layers.Dropout(0.2))
model_selu.add(keras.layers.Dense(units = 8, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.softmax))

model_selu.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

Το προηγούμενο μοντέλο, αλλά με selu.

In [0]:
selu_history = model_selu.fit(Train_Data, Train_Labels, epochs = 1000, validation_data = (Test_Data, Test_Labels))

plot_history([('Subset feature vector', relu_history), ('Subset feature vector selu', selu_history)])

Εκπαίδευση μοντέλου με selu για manual split.

In [0]:
loo = LeaveOneOut()

cvscores = []
for train_index, test_index in loo.split(Data):
  model_selu.fit(Data.to_numpy()[train_index], Labels.to_numpy()[train_index], epochs = 1)
  scores = model_selu.evaluate(Data.to_numpy()[test_index], Labels.to_numpy()[test_index])
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Εκπαίδευση μοντέλου με selu για Leave One Out Cross Validation.

In [0]:
kf = ms.KFold(5)
cvscores = []
for train_index, test_index in kf.split(Data):
  model_selu = keras.Sequential()
  
  model_selu.add(keras.layers.Dense(units = 32, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu, input_dim = 32))
  model_selu.add(keras.layers.Dropout(0.2))
  model_selu.add(keras.layers.Dense(units = 16, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.selu))
  model_selu.add(keras.layers.Dropout(0.2))
  model_selu.add(keras.layers.Dense(units = 8, kernel_regularizer=keras.regularizers.l2(0.001), activation = tf.nn.softmax))
  
  model_selu.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
  model_selu.fit(Data.to_numpy()[train_index], Labels.to_numpy()[train_index], epochs = 200)
  scores = model_selu.evaluate(Data.to_numpy()[test_index], Labels.to_numpy()[test_index])
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Μοντέλο selu με KFold cross-validation.